In [35]:
import tweepy
import pandas as pd

In [36]:
# Credentials

consumer_key = 'frxFyJskSPkPvxiMu8NKUfvX8'

consumer_secret = 'tCtujcAsHIZF8TSzDxKCBJbfM75v8vNEzClmuA48nSqpnC0PH0'

access_token = '557446265-KT2R47ApGYDLiLMDjbYTQONxhctzAtdhbZRm39SE'

access_token_secret = 'nYaLC8D3KzHPcKCRNIoNclAEiMDZ5mROmCqpZZsyheAxb'


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [102]:
user_id = []
user_name = []
location = []

name = ['saifuddintasnim', 'JoeBiden', 'ManUtd']

for i in name:
    user = api.get_user(i)
    user_id.append(user.id_str)
    user_name.append(user.name)
    location.append(user.location)
    
profile_info = pd.DataFrame({'user_id': user_id, 'user_name': user_name, 'location': location})

profile_info

,user_id,user_name,location
0,557446265,Saifuddin Tasnim,Singapore
1,939091,Joe Biden,"Washington, DC"
2,558797310,Manchester United,"Old Trafford, Manchester"
